In [2]:
COLORS = {
    'header': '\033[95m',
    'blue': '\033[94m',
    'cyan': '\033[96m',
    'green': '\033[92m',
    'warning': '\033[93m',
    'fail': '\033[91m',
    'endc': '\033[0m',
    'bold': '\033[1m',
    'underline': '\033[4m'
}

def color_text(text: str, color: str) -> str:
    return f"{COLORS.get(color, COLORS['endc'])}{text}{COLORS['endc']}"


## 🕰️ Context & History: from dataclasses → Pydantic

### 1. Why Pydantic was created and what problem it tries to solve

* Pydantic was created by Samuel Colvin out of “frustration and curiosity about type hints”: he wanted to explore whether Python’s type annotations could be used not only for documentation / static analysis, but at runtime — to validate and parse data. ([pydantic.dev][1])
* The essence: Python’s standard type hints and data containers (like dataclasses) do not enforce anything at runtime. If you expect a field to be e.g. `int`, nothing stops someone from assigning a string — until that causes a bug somewhere else. Many projects need **robust validation, parsing, and data integrity**, especially when data comes from external sources (API requests, JSON, databases, user input, etc). Pydantic addresses that gap. ([DataCamp][2])
* Over time, as Python apps got more complex, especially web backends, microservices, APIs, ML systems, etc — data quality and validation became critical. Pydantic gained traction because it turned type hints into **runtime validated, self-documented data models** that integrate nicely with other frameworks. ([docs.pydantic.dev][3])
* The arrival of **Pydantic v2** (released **30-06-2023**). ([pydantic.dev][4])

  * With v2, Pydantic was **rewritten** internally: its core validation engine was extracted into a separate `pydantic-core` written in **Rust**. ([pydantic.dev][5])
  * This rewrite addresses two main goals: **performance** (Rust backend is faster than pure-Python validation) and **cleaner, more maintainable internal architecture** (validators as composable small components). ([pydantic.dev][5])
  * According to the authors / maintainers: this design makes Pydantic v2 much faster than v1 — which is important for production workloads, APIs, microservices, data-heavy apps. ([Medium][6])

### 2. What Pydantic brings beyond dataclasses (i.e. “why not just dataclasses + manual checks”)

Here are the main motivations / added value of Pydantic compared to vanilla dataclasses:

* **Runtime validation and parsing**: when you create a `Pydantic` model from data (e.g. JSON, dict), Pydantic checks types, coerces values when needed (if allowed), and raises errors when data is invalid — instead of silently producing invalid objects. ([docs.pydantic.dev][3])
* **Automatic serialization / deserialization**: Pydantic models have built-in methods to export (e.g. to dict / JSON), and to build models from external data (JSON, dict, nested structures) — no need to manually write parsing logic. ([docs.pydantic.dev][3])
* **Nested schemas, complex types, defaults, optional / required fields, constraints**: Pydantic supports nested models, optional fields, field defaults, type coercion, and can enforce many constraints using type hints + built-in facilities. ([Medium][7])
* **Integration with modern web frameworks & ecosystem**: Pydantic is widely used in API frameworks (for instance FastAPI uses Pydantic models for request/response types), config management, data exchange between services — making data modeling consistent across layers. ([Wikipedia][8])
* **Performance (v2)**: because of the Rust-based core, Pydantic v2 offers better performance for validation/serialization compared to many older Python-only validation libs. ([pydantic.dev][5])

So, while dataclasses are great for simple, internal data containers (where you control data and trust it), Pydantic shines when you need **robustness, safety, data validation, inter-system data exchange, and runtime integrity**.


[1]: https://pydantic.dev/about?utm_source=chatgpt.com "About Pydantic - Our Mission, Team & Story"
[2]: https://www.datacamp.com/tutorial/pydantic?utm_source=chatgpt.com "Pydantic: A Guide With Practical Examples"
[3]: https://docs.pydantic.dev/latest/?utm_source=chatgpt.com "Welcome to Pydantic - Pydantic Validation"
[4]: https://pydantic.dev/articles/pydantic-v2-final?utm_source=chatgpt.com "Announcement: Pydantic V2 Release"
[5]: https://pydantic.dev/articles/pydantic-v2?utm_source=chatgpt.com "Introducing Pydantic v2 - Key Features"
[6]: https://medium.com/codex/migrating-to-pydantic-v2-5a4b864621c3?utm_source=chatgpt.com "Migrating to Pydantic V2"
[7]: https://medium.com/%40hasanmahira/an-introduction-to-pydantic-the-powerful-data-validation-for-your-rest-apis-89a239cbe816?utm_source=chatgpt.com "An Introduction to Pydantic: the powerful Data Validation ..."
[8]: https://en.wikipedia.org/wiki/FastAPI?utm_source=chatgpt.com "FastAPI"

## 🚀 Let's Start Coding with Pydantic (V2)!

As we saw earlier with dataclasses, Pydantic makes defining data models easy and powerful. Let's dive into some code examples to see how Pydantic works in practice!


1. **Basic User Example**:
  * Let's compare a simple dataclass and a Pydantic model for a `User` entity.
  

In [29]:
from dataclasses import dataclass


@dataclass
class UserDataclass:
    id: int
    email: str
    name: str
    age: int


# Example Usage:
user_1 = UserDataclass(id=1, email="neo@email.com", name="Neo", age=27)
user_2 = UserDataclass(id="1", email="neo@email.com", name="Neo", age="27")


print(f"User 1: {color_text(user_1, 'green')}")
print(f"User 2: {color_text(user_2, 'green')}")
print()
print(f"User 01: Id Type: {color_text(type(user_1.id), 'cyan')}, Age Type: {color_text(type(user_1.age), 'cyan')}")
print(f"User 02: Id Type: {color_text(type(user_2.id), 'cyan')}, Age Type: {color_text(type(user_2.age), 'cyan')}")
print()
similar_object = (user_1 == user_2)
print(f"User 01 and User 02 Equality Check: {color_text(similar_object, 'fail' if not similar_object else 'green')}")


User 1: UserDataclass(id=1, email='neo@email.com', name='Neo', age=27)
User 2: UserDataclass(id='1', email='neo@email.com', name='Neo', age='27')

User 01: Id Type: <class 'int'>, Age Type: <class 'int'>
User 02: Id Type: <class 'str'>, Age Type: <class 'str'>

User 01 and User 02 Equality Check: False


In [59]:
from pydantic import BaseModel


class UserPydantic(BaseModel):
    id: int
    email: str
    name: str
    age: int

# Example Usage:
user_3 = UserPydantic(id=1, email="neo@email.com", name="Neo", age=27)
user_4 = UserPydantic(id="1", email="neo@email.com", name="Neo", age="27")

print()
print(f"User 3: {color_text(user_3, 'green')}")
print(f"User 4: {color_text(user_4, 'green')}")
print()
print(f"User 03: Id Type: {color_text(type(user_3.id), 'cyan')}, Age Type: {color_text(type(user_3.age), 'cyan')}")
print(f"User 04: Id Type: {color_text(type(user_4.id), 'cyan')}, Age Type: {color_text(type(user_4.age), 'cyan')}")
print()
similar_object = (user_3 == user_4)
print(f"User 03 and User 04 Equality Check: {color_text(similar_object, 'fail' if not similar_object else 'green')}")

try:
    user_5 = UserPydantic(id="invalid_id", email="neo@email.com", name="Neo", age="27")
except Exception as e:
    print()
    print(f"{color_text('Error Creating User 5', 'warning')}: \n{color_text(str(e), 'fail')}")




User 3: id=1 email='neo@email.com' name='Neo' age=27
User 4: id=1 email='neo@email.com' name='Neo' age=27

User 03: Id Type: <class 'int'>, Age Type: <class 'int'>
User 04: Id Type: <class 'int'>, Age Type: <class 'int'>

User 03 and User 04 Equality Check: True

Error Creating User 5: 
1 validation error for UserPydantic
id
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='invalid_id', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/int_parsing


**What changes / what Pydantic does**:

* Even if we pass strings `"1"` or `"27"` to fields annotated as `int`, Pydantic will try to **coerce** them into ints (by default), so `u2.id == 1` and `u2.age == 27`. This gives flexibility when parsing data from JSON or APIs.
* If input data cannot be coerced (e.g. `"abc"` for `int`), Pydantic will raise a `ValidationError`, preventing creation of invalid objects — that’s a major benefit vs dataclasses (which would happily accept wrong types).
* The model inherits from `BaseModel`, which provides many capabilities out of the box (validation, serialization, nested models, etc.).

This is powerful especially in “boundary” contexts (parsing user input, JSON payloads, external data) where we can’t trust data to be correct.


2. **Nested object example (User + Address)**:
    * Let's see how Pydantic handles nested models compared to dataclasses.
    

In [41]:
from dataclasses import dataclass


@dataclass
class AddressDataclass:
    street: str
    city: str
    zip_code: str

@dataclass
class UserDataclassNested:
    id: int
    email: str
    name: str
    age: int
    address: AddressDataclass

# Example Usage:
address = AddressDataclass(
    street="Baker Street, 221B",
    city="London",
    zip_code="W1U 6SG",
)

user = UserDataclassNested(
    id=1,
    email="sherlock@email.com",
    name="Sherlock Holmes",
    age=40,
    address=address,
)

print(f"Dataclass User with Nested Address: \n{color_text(user, 'green')}")
print(f"User Dataclass City: {color_text(user.address.city, 'cyan')}")


Dataclass User with Nested Address: 
UserDataclassNested(id=1, email='sherlock@email.com', name='Sherlock Holmes', age=40, address=AddressDataclass(street='Baker Street, 221B', city='London', zip_code='W1U 6SG'))
User Dataclass City: London


In [39]:
from pydantic import BaseModel


class AddressPydantic(BaseModel):
    street: str
    city: str
    zip_code: str

class UserPydanticNested(BaseModel):
    id: int
    email: str
    name: str
    age: int
    address: AddressPydantic

# Example Usage:
data = {
    "id": "1",
    "email": "sherlock@email.com",
    "name": "Sherlock Holmes",
    "age": "40",
    "address": {
        "street": "Baker Street, 221B",
        "city": "London",
        "zip_code": "W1U 6SG",
    },
}

user_pydantic = UserPydanticNested( **data )

print(f"Pydantic User with Nested Address: \n{color_text(user_pydantic, 'green')}")
print(f"User Pydantic City: {color_text(user_pydantic.address.city, 'cyan')}")


Pydantic User with Nested Address: 
id=1 email='sherlock@email.com' name='Sherlock Holmes' age=40 address=AddressPydantic(street='Baker Street, 221B', city='London', zip_code='W1U 6SG')
User Pydantic City: London


**Benefits compared with dataclasses**:

* Automatically handles nested data: even when JSON/dict uses nested structure, Pydantic will parse and instantiate nested models correctly.
* Coercion: even if `id` and `age` come as strings, they’ll be converted.
* Validation: if some field is missing, or types incompatible, you get an explicit error — safer than silent failures.
* Serialization: you can easily convert back to dict or JSON (with correct types), ready for output / storage / API response.


In [44]:
user_pydantic

UserPydanticNested(id=1, email='sherlock@email.com', name='Sherlock Holmes', age=40, address=AddressPydantic(street='Baker Street, 221B', city='London', zip_code='W1U 6SG'))

In [43]:
user_pydantic.model_dump()

{'id': 1,
 'email': 'sherlock@email.com',
 'name': 'Sherlock Holmes',
 'age': 40,
 'address': {'street': 'Baker Street, 221B',
  'city': 'London',
  'zip_code': 'W1U 6SG'}}

3. **More complex example: list fields, defaults, optionality**:
  * Let's define a `Product` model with lists, optional fields, and defaults.

In [57]:
from typing import Self, List, Optional
from pydantic import BaseModel, Field


class Product(BaseModel):
    name: str
    price: float
    description: Optional[str] = None

class InventoryModel(BaseModel):
    products: List[Product] = Field(default_factory=list)

    def add_product(self: Self, product: Product) -> None:
        self.products.append(product)

    def list_products(self: Self) -> List[Product]:
        return self.products


# Example Usage:
inventory = InventoryModel()
product = Product(name="Laptop", price=999.99)

print(f"Initial Inventory: \n\t{color_text(inventory, 'green')}")
inventory.add_product(product)
print(f"Inventory after adding product: \n\t{color_text(inventory, 'green')}")

print()

print(f"Serialization into JSON: \n\t{color_text(inventory.model_dump_json(), 'cyan')}")
print(f"Serialization into Dict: \n\t{color_text(inventory.model_dump(), 'cyan')}")


Initial Inventory: 
	products=[]
Inventory after adding product: 
	products=[Product(name='Laptop', price=999.99, description=None)]

Serialization into JSON: 
	{"products":[{"name":"Laptop","price":999.99,"description":null}]}
Serialization into Dict: 
	{'products': [{'name': 'Laptop', 'price': 999.99, 'description': None}]}


**What Pydantic adds / simplifies**:

* Default values and default factories (via `Field`) — e.g. `products` defaults to empty list — avoiding shared mutable defaults issue.
* Optional fields — e.g. `description: Optional[str] = None`.
* Built-in JSON serialization (via `.model_dump_json()`), so you don’t need to manually convert objects to dicts then `json.dumps`.
* Type enforcement/coercion for nested and collection types: ensures the list contains valid `Product` items, and if you parse from raw data, Pydantic performs validation and coercion.


##  What Pydantic Gives Us (Beyond Dataclasses):

The main **conceptual advantages** of using Pydantic vs raw dataclasses (or manual object definitions):

* **`Schema + Validation + Safety by default`**: We define expected structure + types once; model ensures data validity at runtime, avoiding many classes of bugs.
* **`Parsing from external data made easy`**: Great for APIs, JSON responses, configuration files — we don’t write manual parsing logic.
* **`Consistent data representation + serialization`**: Always the same model → easier to dump to JSON/dict, send over network, store, etc.
* **`Cleaner code base in real apps`**: Less “if this or that type, convert / validate manually” logic scattered around: central place (model definition) controls data shape.
* **`Better maintainability & readability`**: model definitions act as documentation, IDEs and type checkers can help, nested structures remain clear and type-aware.
* **`Performance (v2)`**: thanks to Rust-based core, validation and (de)serialization can be quite efficient — important in data-heavy / high-throughput apps.



## ✅ When Pydantic is “Good” vs “Bad” (on its own)

### **When Pydantic is a *Good* choice**

* **Runtime validation & type safety**: Pydantic enforces that data matches the types and constraints you declared — catching invalid data early. This reduces bugs caused by wrong types or malformed inputs. ([realpython.com][1])
* **Automatic parsing/coercion**: When input data comes from external sources (JSON, dicts, strings), Pydantic can convert/coerce types where appropriate (e.g. `"25"` → `int 25`), easing the burden of manual parsing. ([datacamp.com][2])
* **Serialization / deserialization built-in**: Pydantic models easily convert to dicts or JSON (and vice versa), which is extremely convenient when interacting with APIs, external data, or storage. ([Medium][3])
* **Nested models & complex schemas support**: Pydantic handles nested data structures, collections, optional fields, default values, validators — useful for real-world data modeling beyond flat objects. ([docs.pydantic.dev][4])
* **Error messages & debugging support**: When validation fails, Pydantic gives informative error messages about what failed and why — easier to debug and safer than trusting raw dicts or manual parsing. ([realpython.com][1])
* **Ecosystem & integration**: Pydantic integrates well with many frameworks (web frameworks, config management, data ingestion pipelines), making it practical for production-grade applications. ([Medium][5])
* **Performance (v2)**: Because Pydantic v2 uses a fast, compiled core (Rust-based), validation and (de)serialization are relatively efficient compared to many pure-Python validators, which helps when you have many data transformations. ([docs.pydantic.dev][6])

### **When Pydantic might be a “Bad” / less ideal choice**

* **Overkill for simple or internal-only data containers**: If your use case involves only internal data structures, no external input parsing/validation, or simple DTOs, using Pydantic may add unnecessary overhead and complexity. ([Medium][5])
* **Performance overhead compared to minimal/simple classes**: Because Pydantic does validation, coercion, error checking — object creation is heavier than a plain class or dataclass. For high-throughput, performance-critical internal loops, that overhead may matter. ([news.ycombinator.com][7])
* **External dependency & larger footprint**: Pydantic is a third-party library — this adds dependency management, versioning, and may influence portability. For minimal scripts or libraries, this could be unnecessary.
* **Potential to misuse it — mixing concerns or over-validation**: If you use Pydantic everywhere (even for internal data that come from trusted sources), you may end up with redundant validation, extra complexity, or hampered flexibility. Some authors caution that overusing Pydantic may force Python into behaving more like a strictly typed language, losing some of Python’s flexibility. ([Medium][8])
* **Limited model mapping & interoperability issues**: If you try to convert between Pydantic models and other types (e.g. plain dataclasses, different schema types), sometimes mapping, aliasing or serialization logic becomes complex. ([adaptix][9])
* **Less minimalistic than data containers**: For very simple structural objects (just data, no validation), the built-in simplicity of dataclasses or plain classes may be more appropriate; Pydantic may feel heavy and verbose.

---

## 🔄 Comparison: Pydantic vs Dataclasses — When to Use Each

| Context / Criterion                                                  | **Use Dataclasses**                                                         | **Use Pydantic**                                                                                                         |
| -------------------------------------------------------------------- | --------------------------------------------------------------------------- | ------------------------------------------------------------------------------------------------------------------------ |
| Data origin / trust                                                  | Internal data, controlled flow, trusted sources.                            | External input (user data, JSON, APIs), untrusted sources.                                                               |
| Need for **validation / type coercion**                              | No — data is trusted and validated elsewhere.                               | Yes — you want runtime checks, robust parsing, guarantee of correct types before use.                                    |
| Serialization / deserialization / JSON / API boundary                | Minimal — maybe manual conversion or not needed.                            | Important — need easy JSON/dict convert + validation + possibly nested models.                                           |
| Performance / lightweight objects / many instantiations              | Yes — dataclasses are lightweight, no third-party dependency.               | Less — Pydantic has overhead due to validation and runtime logic, but acceptable if value of validation outweighs cost.  |
| Code readability, maintainability, explicitness                      | Good — simple field definitions, minimal dependencies.                      | Good (maybe better) — explicit schema + validation + self-documenting, but more boilerplate and library usage.           |
| Growth / complexity of data model (nested, many fields, constraints) | Good up to moderate complexity — but manual constraints require extra code. | Excellent — handles nested models, lists, constraints, optional fields, default values, custom validation automatically. |
| Integration with web frameworks / APIs / settings / external libs    | Might require manual glue code / extra serialization logic.                 | Great out-of-the-box support — many frameworks use Pydantic natively (e.g. for request/response schemas).                |

**Rule of thumb (common in real projects):**

* Use **`dataclasses`** (or plain classes) for *internal*, *trusted*, *simple* objects — fast, light, minimal dependencies; ideal for domain models, internal business logic, when data integrity is not a concern.
* Use **`Pydantic`** for *boundary layers* — where data enters or exits your system: API payloads, config files, external data, serialization/deserialization, user input, etc. Especially when you care about validation, data integrity, robust error handling. ([Medium][5])



[1]: https://realpython.com/python-pydantic/?utm_source=chatgpt.com "Pydantic: Simplifying Data Validation in Python"
[2]: https://www.datacamp.com/tutorial/pydantic?utm_source=chatgpt.com "Pydantic: A Guide With Practical Examples"
[3]: https://bhavikjikadara.medium.com/attrs-vs-pydantic-vs-dataclasses-which-to-use-e282116b1cc7?utm_source=chatgpt.com "Attrs vs Pydantic vs Dataclasses: Which to Use?"
[4]: https://docs.pydantic.dev/latest/concepts/validators/?utm_source=chatgpt.com "Validators"
[5]: https://medium.com/%40hadiyolworld007/python-dataclass-vs-pydantic-how-to-choose-8649dce72d2b?utm_source=chatgpt.com "Python Dataclass vs Pydantic: How to Choose"
[6]: https://docs.pydantic.dev/latest/why/?utm_source=chatgpt.com "Why use Pydantic Validation?"
[7]: https://news.ycombinator.com/item?id=40861397&utm_source=chatgpt.com "what are the pros and cons with pydantic models vs data ..."
[8]: https://medium.com/motleycrew-ai/why-too-much-pydantic-can-be-a-bad-thing-892467155cfc?utm_source=chatgpt.com "Why too much Pydantic can be a bad thing | by MotleyCrew"
[9]: https://adaptix.readthedocs.io/en/latest/why-not-pydantic.html?utm_source=chatgpt.com "Why not Pydantic? - adaptix documentation"
